<a href="https://colab.research.google.com/github/jyotidabass/Creating-chatbot-using-tensorflow/blob/main/Creating_chatbot_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization

# Dummy data for finance application
intents = [
    {"tag": "greeting", "patterns": ["Hi", "Hey", "Hello"], "responses": ["Hi, how can I assist you?", "Hello, what can I help you with?"]},
    {"tag": "stock_price", "patterns": ["What is the stock price of Apple?", "Apple stock price"], "responses": ["The current stock price of Apple is $100", "The stock price of Apple is $100"]},
    {"tag": "portfolio", "patterns": ["What is my portfolio?", "My portfolio"], "responses": ["Your portfolio consists of Apple, Google, and Amazon stocks", "You have invested in Apple, Google, and Amazon stocks"]},
]

# Preprocessing data
tags = [intent["tag"] for intent in intents]
patterns = []
responses = []

for intent in intents:
    patterns.extend(intent["patterns"])
    responses.extend(intent["responses"])

# Convert data to numerical values
patterns = np.array(patterns)
responses = np.array(responses)

# Create a dictionary to map tags to numerical values
tag_values = {tag: i for i, tag in enumerate(tags)}

# Restructure tags to match patterns
tags_for_patterns = []
for intent in intents:
    for _ in intent["patterns"]:
        tags_for_patterns.append(tag_values[intent["tag"]])

tags = np.array(tags_for_patterns)

# Split data into training and testing sets
train_patterns, test_patterns, train_tags, test_tags = train_test_split(patterns, tags, test_size=0.2, random_state=42)

# One-hot encoding for tags
# Fixed: Specify num_classes for to_categorical
train_tags = to_categorical(train_tags, num_classes=len(tag_values))
test_tags = to_categorical(test_tags, num_classes=len(tag_values))

# Create a TextVectorization layer
max_tokens = 1000
vectorizer = TextVectorization(max_tokens=max_tokens, output_sequence_length=10)

# Adapt the vectorizer to your training data
vectorizer.adapt(train_patterns)

# Vectorize your training and testing patterns
train_patterns_vec = vectorizer(train_patterns)
test_patterns_vec = vectorizer(test_patterns)

# Create the model
model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(train_patterns_vec.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(len(tag_values), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(train_patterns_vec, train_tags, epochs=50, batch_size=32, validation_data=(test_patterns_vec, test_tags))

# Define a function to predict the tag
def predict_tag(pattern):
    pattern_vec = vectorizer(np.array([pattern]))
    prediction = model.predict(pattern_vec)
    return np.argmax(prediction)

# Define a function to get the response
def get_response(tag, patterns):
    for intent in intents:
        if intent["tag"] == list(tag_values.keys())[list(tag_values.values()).index(tag)]:
            return np.random.choice(intent["responses"])
    return "I didn't understand that."

# Test the chatbot
while True:
    user_input = input("User: ")
    if user_input.lower() == "quit":
        print("Chatbot: Goodbye!")
        break
    tag = predict_tag(user_input)
    response = get_response(tag, patterns)
    print("Chatbot: ", response)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4000 - loss: 1.5539 - val_accuracy: 1.0000 - val_loss: 1.0090
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4000 - loss: 1.2907 - val_accuracy: 1.0000 - val_loss: 1.0128
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4000 - loss: 1.1167 - val_accuracy: 1.0000 - val_loss: 1.0151
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6000 - loss: 1.2382 - val_accuracy: 1.0000 - val_loss: 1.0169
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.2000 - loss: 1.7640 - val_accuracy: 1.0000 - val_loss: 1.0183
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.4000 - loss: 1.1105 - val_accuracy: 1.0000 - val_loss: 1.0208
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6000 - loss: 0.6849 - val_accuracy: 1.0000 - val_loss: 1.0233
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0000e+00 - loss: 1.4823 - val_accuracy: 1.0000 - val_loss: 1.024